In [ ]:
# Prepare Dataset
prepare_boss(stego_method="HILL", alpha="0.4", coding="optimal", skip_existing=True)
base_path = "data/boss"
prepare_boss(stego_method="LSBM", alpha="0.4", coding="optimal", skip_existing=True)
base_path = "data/boss"
# sort images in folders
process_csv("split_tr.csv", "split_te.csv", base_path, "HILL",)
process_csv("split_tr.csv", "split_te.csv", base_path, "LSBM",)



In [ ]:
import sealwatch as sw
cover_path="./data/boss/split_tr_cover",
valid_cover_path="./data/boss/split_te_cover",

In [ ]:
# benchmark for hill
valid_stego_path="./data/boss/split_te_HILL_stego",
stego_path="./data/boss/split_tr_HILL_stego",

model_hill = sw.xunet.XuNetTrainer(
    checkpoints_dir="./checkpoints/",
    batch_size=32,
    num_epochs=960,
    lr=0.001,
    factor=0.1,
    patience=30,
    min_lr=1e-6,
)

model_hill.fit(cover_path, stego_path, valid_cover_path, valid_stego_path, 5000, 1000)

In [ ]:
#benchmark for lsbm
valid_stego_path="./data/boss/split_te_LSBM_stego",
stego_path="./data/boss/split_tr_LSBM_stego",

model_lsbm = sw.xunet.XuNetTrainer(
    checkpoints_dir="./checkpoints/",
    batch_size=32,
    num_epochs=960,
    lr=0.001,
    factor=0.1,
    patience=30,
    min_lr=1e-6,
)

model_lsbm.fit(cover_path, stego_path, valid_cover_path, valid_stego_path, 5000, 1000)

acc = model_lsbm.test(valid_cover_path, valid_stego_path, 1000)

print(acc)

In [ ]:
#Test
hill = sw.xunet.XuNetTrainer()
hill.load_model("./checkpoints/best_model_hill.pt")
acc_hill = hill.test(valid_cover_path, "./data/boss/split_te_HILL_stego", 100)


lsbm = sw.xunet.XuNetTrainer()
lsbm.load_model("./checkpoints/best_model_lsbm.pt")
acc_lsbm = lsbm.test(valid_cover_path, "./data/boss/split_te_LSBM_stego", 100)

print(f"Xunet: LSBM: acc = {acc_lsbm}; HILL: acc= {acc_hill}")

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchmetrics import Accuracy
from tqdm import tqdm

# Import the DatasetLoad class and the B0 model
from B0.src.b0 import InvertedResidual, SqueezeExcite, DepthwiseSeparableConv, B0
from B0.dataset_b0 import DatasetLoad 

In [ ]:
def test_model(
    test_cover_path,
    test_stego_path,
    model_path="./checkpoints/model_best.pth",
    batch_size=16,
    test_size=100,
):
    # Check if the model file exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")

    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset and DataLoader
    test_data = DatasetLoad(
        test_cover_path, test_stego_path, test_size, transform=transforms.ToTensor()
    )
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Load model
    model = torch.load(model_path, map_location=device, weights_only=False)  # Load the entire model object
    model.to(device)
    model.eval()

    # Metric
    test_accuracy = Accuracy(task="binary").to(device)

    # Testing loop
    test_loss = 0
    test_accuracy.reset()
    loss_fn = torch.nn.NLLLoss()  # Negative Log-Likelihood Loss
    progress_bar = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in progress_bar:
            images = torch.cat((batch["cover"], batch["stego"]), 0).to(device)
            labels = torch.cat((batch["label"][0], batch["label"][1]), 0).to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            test_accuracy.update(outputs.argmax(dim=1), labels)

            # Update progress bar
            progress_bar.set_postfix(
                loss=loss.item(), acc=test_accuracy.compute().item()
            )

    # Final results
    avg_test_loss = test_loss / len(test_loader)
    final_test_accuracy = test_accuracy.compute().item()

    return avg_test_loss, final_test_accuracy

In [ ]:
loss_hill, acc_hill = test_model(
    test_cover_path="./data/boss/split_te_cover", 
    test_stego_path="./data/boss/split_te_HILL_stego", 
    model_path="./checkpoints/model_best.pth", 
    test_size=100,
    batch_size=4
    )

loss_lsbm, acc_lsbm = test_model(
    test_cover_path="./data/boss/split_te_cover", 
    test_stego_path="./data/boss/split_te_LSBM_stego", 
    model_path="./checkpoints/model_best.pth", 
    test_size=100,
    batch_size=4
    )

print(f"B0: LSBM: acc = {acc_lsbm}; HILL: acc= {acc_hill}")